In [126]:
import pandas as pd
df_train = pd.read_csv('train.csv', encoding='utf-8')
df_test = pd.read_csv('test.csv', encoding='utf-8')

In [127]:
import re
def preprocessor(text):
    text = re.sub('[\W]+', ' ', text.lower())
    return text

In [128]:
df_train['text'] = df_train['text'].apply(preprocessor)

In [129]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
df_train['author'] = class_le.fit_transform(df_train['author'].values)

In [130]:
from sklearn.model_selection import train_test_split
X, y = df_train.iloc[:, df_train.columns.get_loc('text')].values, \
       df_train.iloc[:, df_train.columns.get_loc('author')].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.3,
                                                   random_state=42,
                                                   stratify=y)

In [131]:
def tokenizer(text):
    return text.split()

In [132]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [133]:
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
def tokenizer_lemma(text):
    return [lemma.lemmatize(word) for word in text.split()]

In [134]:
import pickle
import os
bag_1gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'bag_1gram_lr_pipe.pkl'), 'rb'))
bag_2gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'bag_2gram_lr_pipe.pkl'), 'rb'))

tfidf_1gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'tfidf_1gram_lr_pipe.pkl'), 'rb'))
tfidf_2gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'tfidf_2gram_lr_pipe.pkl'), 'rb'))
tfidf_3gram_lr_pipe = pickle.load(open(os.path.join('pkl_objects', 'tfidf_3gram_lr_pipe.pkl'), 'rb'))

In [135]:
from sklearn.ensemble import VotingClassifier

voting_hard_clf = VotingClassifier(
                                estimators=[
                                    ('bag_1gram', bag_1gram_lr_pipe), 
                                    ('bag_2gram', bag_2gram_lr_pipe), 
                                    ('tfidf_1gram', tfidf_1gram_lr_pipe),
                                    ('tfidf_2gram', tfidf_2gram_lr_pipe),
                                    #('tfidf_3gram', tfidf_3gram_lr_pipe)
                                ],
                                voting='hard')

voting_soft_clf = VotingClassifier(
                                estimators=[
                                    ('bag_1gram', bag_1gram_lr_pipe), 
                                    ('bag_2gram', bag_2gram_lr_pipe), 
                                    ('tfidf_1gram', tfidf_1gram_lr_pipe),
                                    ('tfidf_2gram', tfidf_2gram_lr_pipe),
                                    #('tfidf_3gram', tfidf_3gram_lr_pipe)
                                ],
                                voting='soft')

voting_hard_clf.fit(X_train, y_train)
voting_soft_clf.fit(X_train, y_train)

trained_clf = [bag_1gram_lr_pipe, 
               bag_2gram_lr_pipe, 
               tfidf_1gram_lr_pipe, 
               tfidf_2gram_lr_pipe]
               #tfidf_3gram_lr_pipe]

vote_clf = [voting_hard_clf, 
            voting_soft_clf]

all_clf = trained_clf + vote_clf

In [136]:
from sklearn.metrics import accuracy_score
for clf in (all_clf):
    if clf not in trained_clf:
        clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(accuracy_score(y_test, y_pred))

0.7008852570650324
0.5929519918283963
0.785665645216207
0.5929519918283963


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6074225400068096
0.7301668369084099


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
